In [1]:
import numpy as np
import scipy as sp
from scipy.sparse.linalg import LinearOperator

import MathFunctions as MF
import MPSOperators as MO
import SingleVUMPS as SV

In [2]:
# Line 7 in Algorithm 2
# Calculate updated C from Eq. (16)
def Simple_Next_C(C,AR,AL,HR,HL,h,dtype):
    M = AR.shape[0]; D = AR.shape[1]
    HC = Simple_EffectiveHamiltonian_HC(AR,AL,HR,HL,h)
    val,vec = sp.sparse.linalg.eigs(HC,k=1,which="SR",v0=C)
    if ( dtype == np.dtype("float") ): vec = vec.real
    return vec.reshape(M,M)

# Line 5 in Algorithm 2
# Eq. (10), Eq. (16)
def Simple_EffectiveHamiltonian_HC(AR,AL,HR,HL,h):
    M = AR.shape[0];
    Block1 = np.einsum("efc,eha,dgj,bij,fghi",AL,np.conj(AL),AR,np.conj(AR),h).reshape(M*M,M*M)
    Block2 = np.einsum("ab,cd -> bdac",HL,np.eye(M)).reshape(M*M,M*M)
    Block3 = np.einsum("ab,cd -> bdac",np.eye(M),HR).reshape(M*M,M*M)
    return Block1 + Block2 + Block3

def Next_C(C,AR,AL,HR,HL,h,dtype):
    M = AR.shape[0]; D = AR.shape[1]
    ALAL = np.tensordot(AL,np.conj(AL),([0],[0]))
    ALALh = np.tensordot(ALAL,h,([0,2],[0,2]))
    ARAR = np.tensordot(AR,np.conj(AR),([2],[2]))
    C_ope = Next_C_ope(HR,HL,ARAR,ALALh,dtype)
    val,vec = sp.sparse.linalg.eigs(C_ope,k=1,v0=C.reshape(M*M))
    if ( dtype == np.dtype("float") ):
        vec = vec.real; val = val.real
    vec /= vec[0,0]/np.abs(vec[0,0])
    C_next = vec.reshape(M,M)
    return C_next
    
class Next_C_ope(sp.sparse.linalg.LinearOperator):
    def __init__(self,HR,HL,ARAR,ALALh,dtype):
        self.HR = HR
        self.HL = HL
        self.ARAR = ARAR
        self.ALALh = ALALh
        self.M = HR.shape[0]
        self.shape = [self.M * self.M, self.M * self.M]
        self.dtype = dtype
    def _matvec(self,Cvec):
        C_next = EffectiveHamiltonian_HC(Cvec.reshape(self.M,self.M),self.HR,self.HL,self.ARAR,self.ALALh)
        return C_next    
    
def EffectiveHamiltonian_HC(C,HR,HL,ARAR,ALALh):
    ALALhC = np.tensordot(ALALh,C,([0],[0]))
    Block1 = np.tensordot(ALALhC,ARAR,([3,1,2],[0,1,3]))
    Block2 = np.tensordot(HL,C,([0],[0]))
    Block3 = np.tensordot(C,HR,([1],[0]))
    return Block1 + Block2 + Block3

In [4]:
D = 3; M = 10
#dtype = np.dtype("float"); A = np.random.rand(M,D,M)
dtype = np.dtype("complex"); A = np.random.rand(M,D,M) + 1j * np.random.rand(M,D,M); 
Sx,Sy,Sz,Su,Sd = MF.Spin(D)
h = - ( np.kron(Sx,Sx) + np.kron(Sy,Sy) - np.kron(Sz,Sz) ).real.reshape(D,D,D,D)
AC,C,AR,AL = MO.MixedCanonicalForm(A,dtype)
HR = np.random.rand(M,M); HL = np.random.rand(M,M) # initial HR and HL
for i in range (60):
    HR,er = SV.Calc_HR(AR,HR,h,dtype)
    HL,el = SV.Calc_HL(AL,HL,h,dtype)
    AC = SV.Next_AC(AC,AR,AL,HR,HL,h,dtype)
    C = SV.Next_C(C,AR,AL,HR,HL,h,dtype)
    AR = SV.Next_AR_SVD(AC,C)
    #AR = SV.Next_AR_PolarDecomposition(AC,C)
    AL = SV.Next_AL_SVD(AC,C)
    #AL = SV.Next_AL_PolarDecomposition(AC,C)
    B = SV.Calc_B(AC,C,AR,AL)
    print (er,el,B)

(-0.8973682642639585-2.1824989732133204e-16j) (-0.8973682642639582-9.137113808621589e-17j) 0.06267719385268143
(-1.3708911063330824-4.85722573273506e-17j) (-1.3652231721778398+1.214306433183765e-17j) 0.021817406392028775
(-1.3914204880531509-4.85722573273506e-17j) (-1.390764057019688+3.469446951953614e-17j) 0.013538504600739118
(-1.397712403305342-1.3530843112619095e-16j) (-1.397650350931512+1.9949319973733282e-17j) 0.008415534762343399
(-1.400212651957932-1.4224732503009818e-16j) (-1.4001305780326085+9.020562075079397e-17j) 0.0050608287444869875
(-1.4009054382227732+1.0408340855860843e-17j) (-1.400934445770042-1.231653667943533e-16j) 0.003345069517394948
(-1.4010936610102283-8.847089727481716e-17j) (-1.4010770903872585-3.469446951953614e-18j) 0.0027621271163124177
(-1.4011140083659193-3.469446951953614e-18j) (-1.4011235029157523-9.367506770274758e-17j) 0.0025201015336673667
(-1.4011321390710099+8.500145032286355e-17j) (-1.401125528845224-5.898059818321144e-17j) 0.002362765473416468
(-